<a href="https://colab.research.google.com/github/SarangaSiriwardhana9/-Procurement-for-Construction-Industry-/blob/main/protection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
!pip install pandas openpyxl scikit-learn


In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, hamming_loss

# Load the Excel file
df = pd.read_excel("Weather Data.xlsx")
print(df.head())

        Date  Location  Rainfall (mm)  Min Temp (°C)  Max Temp (°C)  \
0 2023-01-01  PUTTALAM            0.0           21.5           31.6   
1 2023-01-02  PUTTALAM            0.0           21.0           31.7   
2 2023-01-03  PUTTALAM            0.0           20.5           31.8   
3 2023-01-04  PUTTALAM            0.0           20.4           31.4   
4 2023-01-05  PUTTALAM            0.0           21.9           32.7   

  Watering Frequency  Water Used (L/m²)    Fertilizer Type  Fertilizer (g/m²)  \
0       Watered once                  5  Gliricidia leaves                700   
1       Watered once                  5                NaN                  0   
2       Watered once                  5                NaN                  0   
3       Watered once                  5                NaN                  0   
4       Watered once                  5                NaN                  0   

  Crop Stage Protection Method  
0        0-3             MU,FW  
1        0-3        

In [118]:
# Select relevant features
features = ['Rainfall (mm)', 'Min Temp (°C)', 'Max Temp (°C)', 'Location']
X = df[features]

# Encode 'Location' as categorical data
X['Location'] = X['Location'].map({'PUTTALAM': 0, 'KURUNEGALA': 1})

# Prepare the target variable (Protection Method)
y = df['Protection Method'].str.split(',')

# Use MultiLabelBinarizer to convert protection methods to binary format
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

print("Features shape:", X.shape)
print("Target shape:", y_encoded.shape)
print("Protection methods:", mlb.classes_)

Features shape: (1218, 4)
Target shape: (1218, 9)
Protection methods: ['CP' 'DS' 'FW' 'MU' 'NP' 'SB' 'SH' 'SP' 'WB']


<ipython-input-118-ec4362bea1f4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Location'] = X['Location'].map({'PUTTALAM': 0, 'KURUNEGALA': 1})


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [120]:
# Logistic Regression
lr_model = MultiOutputClassifier(LogisticRegression())
lr_model.fit(X_train, y_train)

# Decision Tree
dt_model = MultiOutputClassifier(DecisionTreeClassifier())
dt_model.fit(X_train, y_train)

# Random Forest
rf_model = MultiOutputClassifier(RandomForestClassifier())
rf_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [121]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    hamming = hamming_loss(y_test, y_pred)
    return accuracy, hamming

print("Logistic Regression:")
lr_accuracy, lr_hamming = evaluate_model(lr_model, X_test, y_test)
print(f"Accuracy: {lr_accuracy:.4f}, Hamming Loss: {lr_hamming:.4f}")

print("\nDecision Tree:")
dt_accuracy, dt_hamming = evaluate_model(dt_model, X_test, y_test)
print(f"Accuracy: {dt_accuracy:.4f}, Hamming Loss: {dt_hamming:.4f}")

print("\nRandom Forest:")
rf_accuracy, rf_hamming = evaluate_model(rf_model, X_test, y_test)
print(f"Accuracy: {rf_accuracy:.4f}, Hamming Loss: {rf_hamming:.4f}")

Logistic Regression:
Accuracy: 0.8730, Hamming Loss: 0.0178

Decision Tree:
Accuracy: 0.9836, Hamming Loss: 0.0027

Random Forest:
Accuracy: 0.9877, Hamming Loss: 0.0023


In [122]:
import joblib

# Assuming Random Forest performed the best
joblib.dump(rf_model, 'protection_method_model.pkl')
joblib.dump(mlb, 'multilabel_binarizer.pkl')

['multilabel_binarizer.pkl']

In [125]:
def predict_protection_methods(model, mlb, rainfall, min_temp, max_temp, location):
    # Prepare input data
    input_data = pd.DataFrame([[rainfall, min_temp, max_temp, location]],
                              columns=['Rainfall (mm)', 'Min Temp (°C)', 'Max Temp (°C)', 'Location'])

    # Make prediction
    prediction = model.predict(input_data)

    # Convert binary prediction back to labels
    predicted_methods = mlb.inverse_transform(prediction)[0]

    return predicted_methods

# Example usage
loaded_model = joblib.load('protection_method_model.pkl')
loaded_mlb = joblib.load('multilabel_binarizer.pkl')

example_rainfall = 3.5
example_min_temp = 21.4
example_max_temp = 31.4
example_location = 0  # 0 for PUTTALAM, 1 for KURUNEGALA

predicted_methods = predict_protection_methods(loaded_model, loaded_mlb,
                                               example_rainfall, example_min_temp,
                                               example_max_temp, example_location)

print("Predicted Protection Methods:", predicted_methods)

Predicted Protection Methods: ('FW', 'MU')


In [126]:
pip install joblib


In [127]:
import joblib

# Save the models
joblib.dump(lr_model, 'logistic_regression_model.joblib')
joblib.dump(dt_model, 'decision_tree_model.joblib')
joblib.dump(rf_model, 'random_forest_model.joblib')

print("Models saved successfully!")


Models saved successfully!
